In [59]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm


from PIL import Image, ImageDraw
import glob
import os
import matplotlib.pyplot as plt
import math 


import scipy.misc as smp

import json
from datetime import datetime

from operator import add
import re

import gmplot
from shapely import geometry

In [78]:
# latitude_list = [ 30.3358376, 30.307977, 30.3216419 ] 
# longitude_list = [ 77.8701919, 78.048457, 78.0413095 ] 

coor_list = np.array([ [ 147.955298998399996, -19.893321306345999 ], \
 [ 148.004100560007117, -19.893050274618691 ], \
 [ 148.004398804194921, -19.939217333149461 ], \
 [ 147.955578599026836, -19.939492752634816 ], \
 [ 147.955298998399996, -19.893321306345999 ] ] )

coor_list3 = np.array( [ [ 148.102643769079208, -19.98497714184467 ], \
                                                [ 148.151567620321799, -19.984663182667305 ], \
                                                [ 148.152253886592121, -20.07718124667651 ], \
                                                [ 148.10329963745869, -20.077498424927406 ], \
                                                [ 148.102643769079208, -19.98497714184467 ] ] )


coor_list2 = np.array([     # proserpine region coordinates 
[ 
   147.9552989984,
   -19.893321306346
],
[ 
   149.00386329475,
   -19.884605137462
],
[ 
   149.01671779369,
   -20.876176661406
],
[ 
   147.96143100666,
   -20.885366094496
],
[ 
   147.9552989984,
   -19.893321306346
]])



latitude_list = coor_list3[:,1].tolist()    
longitude_list = coor_list3[:,0].tolist()

# latitude_list = [-19.893321306346]#,-19.89305027461869,-19.93921733314946,-19.939492752634816,-19.893321306346]
# longitude_list = [147.9552989984]#,148.00410056000712,148.00439880419492,147.95557859902684,147.9552989984]

# gmap5 = gmplot.GoogleMapPlotter(30.3164945, 78.03219179999999, 13) 

gmap5 = gmplot.GoogleMapPlotter(-19.893321306345999,147.955298998399996,13) 
  
gmap5.scatter( latitude_list, longitude_list, '# FFFF00', size = 600, marker = False) 
  
# # polygon method Draw a polygon with 
# # the help of coordinates 
gmap5.polygon(latitude_list, longitude_list, color = 'cornflowerblue') 
  
gmap5.draw( "./maptest.html" ) 




In [26]:
10240/512

20.0

In [27]:
# include functions 
%run phase02-mask-generation/SugarUtils.py

./tiles/tile-x7620-y7600.png
./geometries/geo-x7620-y7600.geojson
./mask/mask-x7620-y7600.png


In [76]:
%run phase02-mask-generation/GeoLibrary.py 

We are here
The Bearing is 89.67191206512382
The Distance is 109809.99554660093
The Distance is 109784.18695433934
The Area is 12055.401080543206
Lat long of a  (-19.893321306346, 147.9552989984)
Lat long of b  (-19.884605137462, 149.00386329475)
Testing bearing alternat  19 53m 4.39153s S, 149 0m 13.9055s E


In [108]:
a = [ [ 148.448118224339254, -20.306292582809959 ], \
 [ 148.497045787660284, -20.305881928257023 ], \
 [ 148.497490940813464, -20.352048742575484 ], \
 [ 148.448548584860021, -20.352460106070474 ], \
 [ 148.448118224339254, -20.306292582809959 ] ]

lat1,lon1 = -19.893321306346,147.9552989984   # (0,0)
lat2,lon2 = -19.98497714184467,148.102643769079208
lat3,lon3 = -20.322048742575484,148.497490940813464
lat4,lon4 = -20.306292582809959,148.448118224339254

def latlon_to_xy(lat1,lon1,lat2,lon2):
    dx = abs(lon1-lon2)*40000*math.cos((lat1+lat2)*math.pi/360)/360
    dy = abs(lat1-lat2)*40000/360
    print(dx*1000/10, " ",dy*1000/10)   # answer in km
    return [math.floor(dx*1000/10), math.floor(dy*1000/10)]

print( latlon_to_xy(lat1,lon1,lat3,lon3) )
print( latlon_to_xy(lat1,lon1,lat4,lon4) )
# 10m per pixel resolution

5657.159332048983   4763.63818032759
[5657, 4763]
5142.26970062559   4588.569738488443
[5142, 4588]


In [87]:
config = ConfigParser.ConfigParser()
config.read('geo.config')
%run phase02-mask-generation/GenerateGeoJSON.py

Generating files ......
File Generation completed


In [38]:
# get all xy positions 
geojson_files = f"./Phase02-DataDelivery/geometries/*.geojson"
geojson_names = glob.glob(geojson_files)

# get all the x, y positions 
xy_pair = []
for gf in geojson_names:
    mask_list = re.findall(r'\d+', gf)
    xy_pair.append( (int(mask_list[1]), int(mask_list[2])) )

In [40]:
xy_pair_sorted = sorted(xy_pair, key=lambda tup: (tup[0],tup[1]))

In [50]:
coor_dict = {}
for gf in geojson_names:
    mask_list = re.findall(r'\d+', gf)
    x,y = int(mask_list[1]), int(mask_list[2])
    with open(gf) as f:
        jo = json.load(f)
    coor_list = jo["features"][0]["geometry"]["coordinates"][0]
    coor_dict["x{}-y{}".format(x,y)] = coor_list


In [53]:
# len(coor_dict)  # 441
len(xy_pair_sorted)   #441

441

In [56]:
def get_num_pixels(filepath):
    width, height = Image.open(filepath).size
    return [width, height]

print(get_num_pixels("./phase02-mask-generation/FullImage.png"))

[10980, 10980]


In [72]:
with open("./phase02-mask-generation/FullGeometry.json") as f:
    fulljson = json.load(f)

fullgo = geometry.GeometryCollection([geometry.shape(fulljson["geometry"]).buffer(0)])

for gf in geojson_names:
    with open(gf) as f:
        jo = json.load(f)
    geo_json_features = jo["features"]
#     tile=geometry.GeometryCollection(geometry.shape(coor_list).buffer(0))
    tile=geometry.GeometryCollection([geometry.shape(feature["geometry"]).buffer(0) for feature in geo_json_features])
    print( tile) 
    if tile.intersects(fullgo):
        print(True)
        print( fullgo.intersection(tile) )
    break


GEOMETRYCOLLECTION (POLYGON ((148.156346412259 -20.63229905104974, 148.2053849264256 -20.63196798736892, 148.2057409032773 -20.67813555888257, 148.1566875594567 -20.67846658867755, 148.156346412259 -20.63229905104974)))
True
POLYGON ((148.156346412259 -20.63229905104974, 148.2053849264256 -20.63196798736892, 148.2057409032773 -20.67813555888257, 148.1566875594567 -20.67846658867755, 148.156346412259 -20.63229905104974))


In [74]:
mask_files = f"./Phase02-DataDelivery/masks/*.png"
mask_images = glob.glob(mask_files)

# get all the x, y positions of tiles from masks file name
masked_xy_pair = []
for mask_img in mask_images:
    mask_list = re.findall(r'\d+', mask_img)
    masked_xy_pair.append( (mask_list[1],mask_list[2]) )
    
masked_xy_pair_sorted = sorted(masked_xy_pair, key=lambda tup: (tup[0],tup[1]))

In [77]:
masked_xy_pair_sorted

[('1536', '1024'),
 ('1536', '1536'),
 ('2048', '1024'),
 ('2048', '1536'),
 ('4608', '4608'),
 ('4608', '5120'),
 ('4608', '5632'),
 ('4608', '6144'),
 ('5120', '3584'),
 ('5120', '4096'),
 ('5120', '4608'),
 ('5120', '5120'),
 ('5120', '5632'),
 ('5120', '6144'),
 ('5120', '6656'),
 ('5120', '7168'),
 ('5632', '4096'),
 ('5632', '4608'),
 ('5632', '5120'),
 ('5632', '5632'),
 ('5632', '6144'),
 ('5632', '6656'),
 ('5632', '7168'),
 ('5632', '7680'),
 ('5632', '8192'),
 ('5632', '8704'),
 ('6144', '4096'),
 ('6144', '4608'),
 ('6144', '5120'),
 ('6144', '5632'),
 ('6144', '6144'),
 ('6144', '6656'),
 ('6144', '7168'),
 ('6144', '7680'),
 ('6144', '8192'),
 ('6144', '8704'),
 ('6144', '9216'),
 ('6144', '9728'),
 ('6656', '10240'),
 ('6656', '4096'),
 ('6656', '4608'),
 ('6656', '5120'),
 ('6656', '5632'),
 ('6656', '6144'),
 ('6656', '6656'),
 ('6656', '7168'),
 ('6656', '7680'),
 ('6656', '8192'),
 ('6656', '8704'),
 ('6656', '9216'),
 ('6656', '9728'),
 ('7168', '10240'),
 ('7168', 

In [ ]:
[[ 148.102972282445677, -20.031237148182278 ], \
 [ 148.151910966339955, -20.030921304516287 ], \
 [ 148.152253886592121, -20.07718124667651 ], \
 [ 148.10329963745869, -20.077498424927406 ], \
 [ 148.102972282445677, -20.031237148182278 ] ] 


In [121]:
inputGeo = { "type": "Polygon", "coordinates": [[ [ 148.448118224339254, -20.306292582809959 ], \
                                                 [ 148.497045787660284, -20.305881928257023 ], \
                                                 [ 148.497490940813464, -20.322048742575484 ], \
                                                 [ 148.448548584860021, -20.322460106070474 ], \
                                                 [ 148.448118224339254, -20.306292582809959 ] ]]  }
inputGeo_go = geometry.GeometryCollection([geometry.shape(inputGeo).buffer(0)])

# find the max topleft, max bottom right
coordinates_list = np.array( inputGeo["coordinates"][0] )

print( np.max(coordinates_list, axis=0) )
print( np.min(coordinates_list, axis=0) )

"""
SUGAR_JSON = "./phase02-mask-generation/FullSugar.geojson"
with open(SUGAR_JSON) as f:
    features = json.load(f)["features"]


top_left = inputGeo["coordinates"][0][0]
bottom_right = inputGeo["coordinates"][0][2]
lat2,lon2 = top_left[1], top_left[0]
lat3,lon3 = bottom_right[1], bottom_right[0]
lat1,lon1 = -19.893321306346,147.9552989984   # (0,0) the origin
[top_x,top_y] = latlon_to_xy(lat1,lon1,lat2,lon2)
[bottom_x,bottom_y] = latlon_to_xy(lat1,lon1,lat3,lon3)

print([top_x, top_y])
print([bottom_x,bottom_y])


# for each masked tile files
xy_tile_needed_ij_dict = {}
# for xy in tqdm(masked_xy_pair):
    
#     x,y = int(xy[0]),int(xy[1])
#     geoJSONPath = "./Phase02-DataDelivery/geometries/geo-x{}-y{}.geojson".format(x,y)
#     with open(geoJSONPath) as f:
#         geo_json_features = json.load(f)["features"]
#     tile=geometry.GeometryCollection([geometry.shape(feature["geometry"]).buffer(0) for feature in geo_json_features])

    
result=geometry.GeometryCollection()   # sugarcane masks that intersects tile
for k in features:
    if geometry.shape(k["geometry"]).intersects(inputGeo_go):
        result=result.union(geometry.shape(k["geometry"]).intersection(inputGeo_go))


needed_ij_list = []
if inputGeo_go.intersects( result ):
    # find the mask in the input region
    for i in tqdm(list(range(top_x, bottom_x))):
        for j in list(range(top_y, bottom_y)):
            lat_long=GetLatLongForCoords(i,j)
            if result.intersects(geometry.Point(lat_long)):
                # this coordinate is needed
                needed_ij_list.append((i,j))
    
    
        # this tile is needed 
#         for i in list(range(0,512)):
#             for j in list(range(0,512)):
#                 nx = x+i
#                 ny = y+j
#                 lat_long=GetLatLongForCoords(nx,ny)
#                 if result.intersects(geometry.Point(lat_long)) and inputGeo_go.intersects(geometry.Point(lat_long)):
#                     # this coordinate is needed
#                     needed_ij_list.append((i,j))
    
#     xy_tile_needed_ij_dict["x{}-y{}".format(x,y)] = needed_ij_list

"""


[148.49749094 -20.30588193]
[148.44811822 -20.32246011]


'\nSUGAR_JSON = "./phase02-mask-generation/FullSugar.geojson"\nwith open(SUGAR_JSON) as f:\n    features = json.load(f)["features"]\n\n\ntop_left = inputGeo["coordinates"][0][0]\nbottom_right = inputGeo["coordinates"][0][2]\nlat2,lon2 = top_left[1], top_left[0]\nlat3,lon3 = bottom_right[1], bottom_right[0]\nlat1,lon1 = -19.893321306346,147.9552989984   # (0,0) the origin\n[top_x,top_y] = latlon_to_xy(lat1,lon1,lat2,lon2)\n[bottom_x,bottom_y] = latlon_to_xy(lat1,lon1,lat3,lon3)\n\nprint([top_x, top_y])\nprint([bottom_x,bottom_y])\n\n\n# for each masked tile files\nxy_tile_needed_ij_dict = {}\n# for xy in tqdm(masked_xy_pair):\n    \n#     x,y = int(xy[0]),int(xy[1])\n#     geoJSONPath = "./Phase02-DataDelivery/geometries/geo-x{}-y{}.geojson".format(x,y)\n#     with open(geoJSONPath) as f:\n#         geo_json_features = json.load(f)["features"]\n#     tile=geometry.GeometryCollection([geometry.shape(feature["geometry"]).buffer(0) for feature in geo_json_features])\n\n    \nresult=geometr

In [112]:
len(needed_ij_list)

42342

In [116]:
from decimal import Decimal
float(-20.312048742575484)

-20.312048742575485